<a href="https://colab.research.google.com/github/enesergen/TensorflowBeginner/blob/main/TensorflowBeginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q sklearn

In [2]:
%tensorflow_version 2.x #bu satır notebook için gerekli local cihazlarda gerekli değil

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [3]:
from __future__ import absolute_import,division,print_function,unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output #yanlızca notebook için
from six.moves import urllib


In [4]:
import tensorflow as tf

In [5]:
from tensorflow import feature_column #lineer regression algoritması ve modeli için kullanılacak

In [6]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # training data
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

In [7]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()  
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))



In [8]:
def make_input_fn(data_df,label_df,num_epochs=10,shuffle=True,batch_size=32):
  def input_function():
    ds=tf.data.Dataset.from_tensor_slices(dict(data_df),label_df)
    if shuffle:
      ds=ds.shuffle(1000)
    ds=ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn=make_input_fn(dftrain,y_train)#burada bize bir dataset objesi dönüyor bunu modelimizi eğitirken kullanacağız
eval_input_fn=make_input_fn(dfeval,y_eval,num_epochs=1,shuffle=False)#bununla eğitme train işlemi olmadğını için shuffle false ve num 1 veriliyor

In [9]:
#model oluşturma
linear_est=tf.estimator.LinearClassifier(feature_columns=feature_columns)